In [2]:
[
    {
        "Product Name": "Product A",
        "Product Price": "₹1000",
        "Sale Discount": "50%",
        "Best Seller Rating": "4.5",
        "Ship From": "India",
        "Sold By": "Amazon",
        "Rating": "4.8",
        "Product Description": "A great product",
        "Number Bought in the Past Month": 1000,
        "Category Name": "Kitchen",
        "All Available Images": ["image1.jpg", "image2.jpg"]
    }
]


[{'Product Name': 'Product A',
  'Product Price': '₹1000',
  'Sale Discount': '50%',
  'Best Seller Rating': '4.5',
  'Ship From': 'India',
  'Sold By': 'Amazon',
  'Rating': '4.8',
  'Product Description': 'A great product',
  'Number Bought in the Past Month': 1000,
  'Category Name': 'Kitchen',
  'All Available Images': ['image1.jpg', 'image2.jpg']}]

In [3]:
import time
import logging
import csv
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Set up logging
logging.basicConfig(level=logging.INFO)

# Initialize WebDriver
def init_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# Function to login into Amazon
def amazon_login(driver, email, password):
    driver.get('https://www.amazon.in/')
    try:
        sign_in_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(text(),'Account & Lists')]"))
        )
        sign_in_button.click()

        email_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_email"))
        )
        email_input.send_keys(email)
        driver.find_element(By.ID, "continue").click()

        password_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "ap_password"))
        )
        password_input.send_keys(password)
        driver.find_element(By.ID, "signInSubmit").click()
    except Exception as e:
        logging.error(f"Error during login: {e}")

# Function to scrape product details from a category
def scrape_category(driver, category_url):
    driver.get(category_url)
    products = []

    try:
        # Wait until the products are visible
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.zg-item-immersion'))
        )

        # Loop through product listings
        product_elements = driver.find_elements(By.CSS_SELECTOR, '.zg-item-immersion')

        for product in product_elements:
            try:
                product_name = product.find_element(By.CSS_SELECTOR, '.p13n-sc-truncated').text
                price = product.find_element(By.CSS_SELECTOR, '.p-price').text
                discount = product.find_element(By.CSS_SELECTOR, '.p-badge').text
                rating = product.find_element(By.CSS_SELECTOR, '.a-icon-alt').text
                image_url = product.find_element(By.CSS_SELECTOR, '.zg-item img').get_attribute('src')
                products.append({
                    "Product Name": product_name,
                    "Product Price": price,
                    "Sale Discount": discount,
                    "Rating": rating,
                    "Image URL": image_url
                })
            except Exception as e:
                logging.error(f"Error extracting product data: {e}")
    
    except Exception as e:
        logging.error(f"Error scraping category: {e}")

    return products

# Save scraped data to CSV
def save_to_csv(data, filename="scraped_data.csv"):
    try:
        keys = data[0].keys()
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=keys)
            writer.writeheader()
            writer.writerows(data)
        logging.info(f"Data saved to {filename}")
    except Exception as e:
        logging.error(f"Error saving data to CSV: {e}")

# Save scraped data to JSON
def save_to_json(data, filename="scraped_data.json"):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        logging.info(f"Data saved to {filename}")
    except Exception as e:
        logging.error(f"Error saving data to JSON: {e}")

if __name__ == "__main__":
    driver = init_driver()

    # Log in (Replace with valid credentials)
    amazon_login(driver, "your_email", "your_password")

    # Scrape data from a sample category
    category_url = "https://www.amazon.in/gp/bestsellers/kitchen/ref=zg_bs_nav_kitchen_0"
    product_data = scrape_category(driver, category_url)

    # Store data in CSV or JSON
    save_to_csv(product_data)
    save_to_json(product_data)

    driver.quit()


INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\User\.wdm\drivers\chromedriver\win64\131.0.6778.204\chromedriver-win32/chromedriver.exe] found in cache
ERROR:root:Error during login: Message: 
Stacktrace:
	GetHandleVerifier [0x0111EC13+23731]
	(No symbol) [0x010AC394]
	(No symbol) [0x00F8BE63]
	(No symbol) [0x00FCFCE6]
	(No symbol) [0x00FCFF2B]
	(No symbol) [0x0100D892]
	(No symbol) [0x00FF1EA4]
	(No symbol) [0x0100B46E]
	(No symbol) [0x00FF1BF6]
	(No symbol) [0x00FC3F35]
	(No symbol) [0x00FC4EBD]
	GetHandleVerifier [0x013FF0D3+3039603]
	GetHandleVerifier [0x01412DEA+3120778]
	GetHandleVerifier [0x0140B592+3089970]
	GetHandleVerifier [0x011B43B0+635984]
	(No symbol) [0x010B4DCD]
	(No symbol) [0x010B2068]
	(No symbol) [0x010B2205]
	(No symbol) [0x010A4FD0]
	BaseThreadInitThunk [0x768E7BA9+25]
	RtlInitializeExceptionChain [0x778AC0CB+107]
	RtlClearBits 